# MLCP with Gurobi

Analiziamo il file in inpput. I file descivono grafi e la prima riga contiene il numero di vertici e di archi, mentre le righe seguenti contengono l'elenco di tutti gli archi.

In [1]:
import numpy as np

In [2]:
def analyse_dimacs(file_path):
    """analyse dataset file
    input: path of the DIMACS file
    output: number of vertices of the graph (int), list of the adges (list of tuple) """
    edges=[]
    with open(file_path, 'r') as file: #open the file in read mode
        num_vertices, num_edges = map(int, file.readline().split())  #firt line
        for line in file:
            u,v=map(int,line.split())
            edges.append((u-1,v-1)) #convert from 1-based to 0-based indices (file parte la numerazione da 1 ma python parte da zero)
            
    return num_vertices, edges

Costruiamo la matrice di adiacenza.

In [3]:
def built_adjacency_matrix (num_vertices, edges):
    """ buil an adjacency matrix from alist of edges
    input: number of vertices of graph, list of edges (u,v) (list of tuple)
    output: adjacency matrix (np.ndarray)"""
    adj_matrix = np.zeros((num_vertices, num_vertices), dtype=int)
    for u,v in edges:
        adj_matrix[u][v]=1
        adj_matrix[v][u]=1 #the graph are undirected
        
    return adj_matrix
    

In [4]:
#aggiungi il tempo così è più faccile fare l'analisi dopo DEVO METTERE ANCHE UN TEMPP LIMITE!! SUL ARTICOLO USANO 12H

In [5]:
import gurobipy as gp
from gurobipy import GRB
import time

In [6]:
import os
mydir = os.getcwd() # path relativo alla cartella corrente
file_path = os.path.join(mydir, '..', 'files', 'queen6_6.col')

n, edges = analyse_dimacs(file_path)
E=built_adjacency_matrix(n, edges)


#print("number of vertices n=",n)

#print("Adjacency matatrix is E=")
#print(E)

In [7]:
analyse_dimacs(file_path)

(36,
 [(0, 7),
  (0, 14),
  (0, 21),
  (0, 28),
  (0, 35),
  (0, 1),
  (0, 2),
  (0, 3),
  (0, 4),
  (0, 5),
  (0, 6),
  (0, 12),
  (0, 18),
  (0, 24),
  (0, 30),
  (1, 8),
  (1, 15),
  (1, 22),
  (1, 29),
  (1, 6),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (1, 7),
  (1, 13),
  (1, 19),
  (1, 25),
  (1, 31),
  (1, 0),
  (2, 9),
  (2, 16),
  (2, 23),
  (2, 7),
  (2, 12),
  (2, 3),
  (2, 4),
  (2, 5),
  (2, 8),
  (2, 14),
  (2, 20),
  (2, 26),
  (2, 32),
  (2, 1),
  (2, 0),
  (3, 10),
  (3, 17),
  (3, 8),
  (3, 13),
  (3, 18),
  (3, 4),
  (3, 5),
  (3, 9),
  (3, 15),
  (3, 21),
  (3, 27),
  (3, 33),
  (3, 2),
  (3, 1),
  (3, 0),
  (4, 11),
  (4, 9),
  (4, 14),
  (4, 19),
  (4, 24),
  (4, 5),
  (4, 10),
  (4, 16),
  (4, 22),
  (4, 28),
  (4, 34),
  (4, 3),
  (4, 2),
  (4, 1),
  (4, 0),
  (5, 10),
  (5, 15),
  (5, 20),
  (5, 25),
  (5, 30),
  (5, 11),
  (5, 17),
  (5, 23),
  (5, 29),
  (5, 35),
  (5, 4),
  (5, 3),
  (5, 2),
  (5, 1),
  (5, 0),
  (6, 13),
  (6, 20),
  (6, 27),
  (6, 34),
  

In [8]:
time_limit=1800
 
#create a new GUrobi problem
mlcp=gp.Model()
mlcp.setParam('outputFlag', 0)
mlcp.modelSense = gp.GRB.MAXIMIZE

#Variables
x = mlcp.addVars(n, vtype=GRB.BINARY, name="x")    # x_i: binary variable for partition
r = mlcp.addVars(n, n, vtype=GRB.BINARY, name="r")  # r_ij: binary variable for inside edge in V_r
b = mlcp.addVars(n, n, vtype=GRB.BINARY, name="b")  # b_ij: binary variable for inside edge in V_b
z = mlcp.addVar(vtype=GRB.INTEGER, name="z")        # z: objective variable (minimum load)



#objective function
mlcp.setObjective(z)


#constraints
for i in range(n):
    for j in range(n):
        mlcp.addConstr(2 * r[i, j] <= x[i] + x[j])   #Constraint 1: r_ij takes value 1 only when both v_i and v_j are in V_r
        mlcp.addConstr(2 * b[i, j] <= 2 - (x[i] + x[j]))  #Constraint 2: b_ij takes value 1 only when both v_i and v_j are in V_b

            
mlcp.addConstr(z <= gp.quicksum(r[i, j] * E[i][j] for i in range(n) for j in range(i+1,n)))  #Constraint 3: z <= number of inside edges conteined in V_r
mlcp.addConstr(z <= gp.quicksum(b[i, j] * E[i][j] for i in range(n) for j in range(i+1,n)))  #Constraint 4: z <= number of inside edges conteined in V_b


# Imposta il limite di tempo
mlcp.setParam("TimeLimit", time_limit)

# Inizia il conteggio del tempo
start_time = time.time()



#solve 
mlcp.optimize()

# Calcola e stampa il tempo di esecuzione
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tempo di esecuzione: {elapsed_time:.2f} secondi")

# Verifica se la soluzione è stata trovata o se il tempo è scaduto
if mlcp.status == gp.GRB.OPTIMAL:
    print("Soluzione ottimale trovata!")
elif mlcp.status == gp.GRB.TIME_LIMIT:
    print("Limite di tempo raggiunto, soluzione non ottimale.")



Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-07
Tempo di esecuzione: 78.03 secondi
Soluzione ottimale trovata!


In [9]:
def display_results(mlcp):
    # Check if the model was solved
    if mlcp.status == gp.GRB.OPTIMAL:
        # Extract and print the optimal objective value
        print(f"Optimal value of z (maximum minimum edges in the same partition): {mlcp.objVal}")
        
        # Print the partition assignments (V_r or V_b)
        print("Partition assignments:")
        for i in range(n):
            partition = "V_r" if x[i].x > 0.5 else "V_b"
            print(f"Vertex {i} is in {partition}")
        
        # Optionally, display edges inside each partition
        print("Edges in V_r:")
        for i in range(n):
            for j in range(n):
                if i != j and r[i, j].x > 0.5:
                    print(f"Edge ({i}, {j}) is inside V_r")
        
        print("Edges in V_b:")
        for i in range(n):
            for j in range(n):
                if i != j and b[i, j].x > 0.5:
                    print(f"Edge ({i}, {j}) is inside V_b")
    else:
        print("Optimal solution not found.")

# Call the function to display results after optimization
display_results(mlcp)


Optimal value of z (maximum minimum edges in the same partition): 91.0
Partition assignments:
Vertex 0 is in V_r
Vertex 1 is in V_r
Vertex 2 is in V_r
Vertex 3 is in V_r
Vertex 4 is in V_r
Vertex 5 is in V_r
Vertex 6 is in V_r
Vertex 7 is in V_r
Vertex 8 is in V_r
Vertex 9 is in V_r
Vertex 10 is in V_r
Vertex 11 is in V_r
Vertex 12 is in V_r
Vertex 13 is in V_r
Vertex 14 is in V_r
Vertex 15 is in V_r
Vertex 16 is in V_r
Vertex 17 is in V_r
Vertex 18 is in V_b
Vertex 19 is in V_b
Vertex 20 is in V_b
Vertex 21 is in V_b
Vertex 22 is in V_b
Vertex 23 is in V_b
Vertex 24 is in V_b
Vertex 25 is in V_b
Vertex 26 is in V_b
Vertex 27 is in V_b
Vertex 28 is in V_b
Vertex 29 is in V_b
Vertex 30 is in V_b
Vertex 31 is in V_b
Vertex 32 is in V_b
Vertex 33 is in V_b
Vertex 34 is in V_b
Vertex 35 is in V_b
Edges in V_r:
Edge (0, 1) is inside V_r
Edge (0, 2) is inside V_r
Edge (0, 3) is inside V_r
Edge (0, 4) is inside V_r
Edge (0, 5) is inside V_r
Edge (0, 6) is inside V_r
Edge (0, 7) is inside V_r
